# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
#numpy helps execute code faster as it is coded in a different language (C++?) oftenly used for finance functions
import numpy as np 
#pandas is short for panel data. helps working with tabular data like Excel
import pandas as pd
#useful for grabbing websites
import requests
#useful for writing excel like tables, formulas, hyperlinks, numbers, etc.
import xlsxwriter
#useful for math formulas and shortcuts
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
#added to eliminate the keyerror where these stocks are delisted
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[501 rows x 1 columns]


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
#sandbox mode under IEX Cloud API helps create a "playground" mode to see if your code is working properly
#sandbox also allows us to have no cost when building this code since the data is FREE but RANDOMIZED
#I also named my secrets.py into a confidential file since apparently there is a secrets module under python now
from confidential import IEX_CLOUD_API_TOKEN


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#this is a response object (39:30)
data = requests.get(api_url).json()
print(data)



{'avgTotalVolume': 90078417, 'calculationPrice': 'iexlasttrade', 'change': 0.05, 'changePercent': 0.00036, 'close': 0, 'closeSource': 'oaflcfii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 146.58, 'iexCloseTime': 1703254601198, 'iexLastUpdated': 1677199681150, 'iexMarketPercent': 0.018936918650070404, 'iexOpen': 146.5, 'iexOpenTime': 1706592576237, 'iexRealtimePrice': 143.66, 'iexRealtimeSize': 101, 'iexVolume': 1341070, 'lastTradeTime': 1671900017056, 'latestPrice': 145.98, 'latestSource': 'IEX Last Trade', 'latestTime': 'June 27, 2022', 'latestUpdate': 1704595986353, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2352440186203, 'oddLotDelayedPrice':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [7]:
#essentially final_dataframe.append is what puts the data under the correct columns. 
#index = my_columns identifies which columns each component of the list in the Series is referring to.
#ignore_index = True is necessary to bypass an error
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,145.98,2352440186203,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    #this is a response object (39:30)
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
    )

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.45,36956940739,N/A
1,AAL,13.79,9171152992,N/A
2,AAP,184.30,11298545631,N/A
3,AAPL,143.36,2401722327015,N/A
4,ABBV,154.51,280400925804,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i+n]
#continue from 58:09 mark

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [] 
#this creates a comma separated list for each symbol group that was separated after each 100 stocks
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True

        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.96,37453216596,N/A
1,AAL,14.06,9200267396,N/A
2,AAP,184.57,11530017045,N/A
3,AAPL,142.38,2347210228296,N/A
4,ABBV,154.10,280218998677,N/A
...,...,...,...,...
496,YUM,121.62,33465510392,N/A
497,ZBH,110.27,23705506211,N/A
498,ZBRA,321.00,16401103787,N/A
499,ZION,53.84,8233698102,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Please try again: ")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [13]:
#since this is equal-weight we just divide the # of shares to buy evenly 
position_size = val / len(final_dataframe.index)
#number_of_apple_shares = position_size / 140
#assuming 140 is the price of apple 
#print(math.floor(number_of_apple_shares))

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.96,37453216596,16
1,AAL,14.06,9200267396,141
2,AAP,184.57,11530017045,10
3,AAPL,142.38,2347210228296,14
4,ABBV,154.10,280218998677,12
...,...,...,...,...
496,YUM,121.62,33465510392,16
497,ZBH,110.27,23705506211,18
498,ZBRA,321.00,16401103787,6
499,ZION,53.84,8233698102,37


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [16]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [17]:
column_formats = {
    'A' : ['Ticker', string_format],
    'B' : ['Stock Price', dollar_format],
    'C' : ['Market Capitalization', dollar_format],
    'D' : ['Numbers of Shares to Buy', integer_format]
}

for clm in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{clm}:{clm}', 20, column_formats[clm][1])
    writer.sheets['Recommended Trades'].write(f'{clm}1', column_formats[clm][0],column_formats[clm][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [18]:
writer.save()